In [ ]:
from MahalanobisRelativeAbundance import MahalanobisRelativeAbundance
import numpy as np
import pandas as pd
import os
import shutil
import util
from sklearn.neural_network import MLPClassifier
from ete3 import NCBITaxa
import warnings
warnings.filterwarnings("ignore")

In [1]:
def taxid_to_lineage(taxid):
    """
    Function for retrieving the taxonomic rank of given taxid
    :param taxid:
    :return:
    """
    desired_ranks = ['phylum', 'class', 'order', 'family', 'genus', 'species']
    ncbi = NCBITaxa()
    lineage = ncbi.get_lineage(taxid)
    rank_to_id = {rank: id for (id, rank) in ncbi.get_rank(lineage).items()}
    rank_to_id = {desired_rank: (rank_to_id[desired_rank] if desired_rank in rank_to_id.keys() else None) for
                  desired_rank in desired_ranks}
    return rank_to_id

aaa
2 3


In [ ]:
def taxonomic_accuracy(prediction, target, use_max=False):
    desired_ranks = ['phylum', 'class', 'order', 'family', 'genus', 'species']
    cnt = np.zeros(7)
    for i in range(prediction.shape[0]):
        if use_max:
            rank_to_id_1 = taxid_to_lineage(host_to_speciesid[idx_to_host_dict[prediction[i, :].argmax()]])
        else:
            rank_to_id_1 = taxid_to_lineage(host_to_speciesid[idx_to_host_dict[prediction[i, :].argmin()]])
        rank_to_id_2 = taxid_to_lineage(host_to_speciesid[idx_to_host_dict[target[i]]])
        for j, rank in enumerate(desired_ranks):
            if rank_to_id_1[rank] == rank_to_id_2[rank] and rank_to_id_1[rank] is not None:
                cnt[j] += 1
    """Strain accuracy is handled separately"""
    if use_max:
        cnt[6] = np.sum(prediction.argmax(axis=1) == target)
    else:
        cnt[6] = np.sum(prediction.argmin(axis=1) == target)
    acc = cnt / prediction.shape[0]
    return acc

In [ ]:
def load_metadata():
    metadata = pd.read_csv('metadata.csv')
    metadata = metadata.dropna()
    metadata = metadata.reset_index(drop=True)

    for i in range(len(metadata.Locus_ID)):
        metadata.at[i, 'Locus_ID'] = metadata.at[i, 'Locus_ID'].split('.')[0]

    metadata.sort_values(by=['Locus_ID'], inplace=True)
    metadata.reset_index(drop=True, inplace=True)
    return metadata

In [ ]:
metadata = load_metadata()
